In [2]:
import pandas as pd
import numpy as np
import multiprocessing
from deap import base,creator,tools
import os
import subprocess
import random
import sys
import tensorflow
import pandas as pd
import numpy as np
import statistics
import tensorflow_addons as tfa 
from sklearn.utils import resample
import tensorflow as tf
import time

2023-08-03 14:43:08.183355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 14:43:08.324883: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 14:43:08.327816: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-03 14:43:08.327826: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [7]:
aa_pos = {"A":0,"C":1,"D":2,"E":3,"F":4,"G":5,"H":6,"I":7,"K":8,"L":9,"M":10,"N":11,"P":12,"Q":13,"R":14,"S":15,"T":16,"V":17,"W":18,"Y":19,"-":20}

def vectorize_peptide(peptide_list): #sparse encoding
    pept_vector = np.zeros((len(peptide_list),100,21,1))
    counter = 0
    counter_list = 0
    for peptide in peptide_list:
        if type(peptide) == float:
            continue
        if len(peptide) > 100:
            peptide = peptide[0:100]
        for i in range(len(peptide)):
            pept_vector[counter_list,i,aa_pos[peptide[i]]]=1
            counter += 1
        for j in range(counter,100):
            pept_vector[counter_list,j,aa_pos["-"]]=1
        counter_list +=1
        counter = 0

    pept_vector = np.squeeze(pept_vector)
    return pept_vector

def threemers(peptide_list): #threemers encoding
    pept_vector = np.zeros((len(peptide_list),8000))
    counter_list = 0
    for peptide in peptide_list:
        counts = {aa_comb: 0 for aa_comb in combinations_list}
        for i in range(len(peptide)-2):
            window = peptide[i:i+3]
            if window in counts:
                counts[window] += 1
        pept_vector[counter_list]=list(counts.values())
        counter_list +=1
    pept_vector = pept_vector.reshape(pept_vector.shape[0],100,80)
    return pept_vector

def mutate_medium(weights_vector): #mutation of the individual, 25% chance for each parameter
    for i in range(len(weights_vector[0])):
        if i < 2:
            probability = np.random.randint(101)
            if probability <= 25: # 25% of probability to mutate
                weights_vector[0][i]=random.uniform(1,10)
        elif 2 <= i < 5:
            probability = np.random.randint(101)
            if probability <= 25: # 25% of probability to mutate
                weights_vector[0][i]=random.randint(5,200)
        else:
            probability = np.random.randint(101)
            if probability <= 25: # 25% of probability to mutate
                weights_vector[0][i]=random.uniform(0.0001,1)
    return weights_vector


def prules(): #rules to create the parameters
    weights=[]
    for i in range(2): #possible weights of the two classe
        weights.append(random.uniform(0.1,10))
    for i in range(3): #kernel size
        weights.append(random.randint(5,500))
    for i in range(6): #learning and deacy rates
        weights.append(random.uniform(0.0001,0.1))
    return weights


def evaluate_medium(weights_vector): #basically the neural network that 
    weights_vector = weights_vector[0]
    save_dir = "/home/baccus/Desktop/PRIN/models/"
    import keras #keras needs to be imported each time otherwise the models will not load properly
    from tensorflow.keras import regularizers
    from keras import models
    from keras import layers
    from keras import regularizers
    from keras import metrics
    tf.config.threading.set_intra_op_parallelism_threads(2)
    tf.config.threading.set_inter_op_parallelism_threads(2)
    randomic_num = random.randint(0,200000000) #da cambiare come vengono registrati i modelli
    METRICS = [keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc',multi_label=True),
      keras.metrics.AUC(name='prc', curve='PR'),
          tfa.metrics.MatthewsCorrelationCoefficient(num_classes = 2,name="MCC")]
    save_dir = "/home/baccus/Desktop/PRIN/models/"       
    functions = peptides_df["Class"]
    peptides_train_60 = peptides_df.sample(frac = 0.6)
    peptides_test_20 = peptides_df.drop(peptides_train_60.index)
    peptides_validate_20 = peptides_test_20.sample(frac = 0.5)
    peptides_test_20 = peptides_test_20.drop(peptides_validate_20.index)
    
    all_data = vectorize_peptide(peptides_df["Sequence"])
    train_data = vectorize_peptide(peptides_train_60["Sequence"])
    test_data = vectorize_peptide(peptides_test_20["Sequence"])
    validation_data = vectorize_peptide(peptides_validate_20["Sequence"])
    all_labels = tensorflow.keras.utils.to_categorical(peptides_df["Class"].tolist(), num_classes=2)
    a = tensorflow.keras.utils.to_categorical(peptides_train_60["Class"].tolist(), num_classes=2)
    b = tensorflow.keras.utils.to_categorical(peptides_test_20["Class"].tolist(), num_classes=2)
    c = tensorflow.keras.utils.to_categorical(peptides_validate_20["Class"].tolist(), num_classes=2)
    checkpoint = keras.callbacks.ModelCheckpoint(save_dir+str(randomic_num)+".hdf5", monitor='val_MCC', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model = models.Sequential()
    weights = {0:weights_vector[0], 1:weights_vector[1]}
    print(test_data[0].shape)
    model.add(layers.Conv1D(weights_vector[3], (4,), kernel_regularizer=regularizers.L2(l2=weights_vector[5]), 
        bias_regularizer=regularizers.L2(l2=weights_vector[6]),activation='relu', input_shape=train_data[0].shape))
    model.add(layers.MaxPooling1D((2,), padding="valid"))
    model.add(layers.Conv1D(weights_vector[4], (4,), activation='relu',kernel_regularizer=regularizers.L2(l2=weights_vector[7]), bias_regularizer=regularizers.L2(l2=weights_vector[8])))
    model.add(layers.MaxPooling1D((2,)))
    model.add(layers.Dropout(weights_vector[9]))
    model.add(layers.Flatten())
    model.add(layers.Dense(weights_vector[2], activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=weights_vector[10])
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=METRICS)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=METRICS)
    model.summary()
    history = model.fit(train_data, a,class_weight=weights, validation_data=(validation_data,c), callbacks=callbacks_list, epochs=50, batch_size=20,use_multiprocessing=False,workers=1,verbose=1)
    his = model.evaluate(test_data,b)
    return (his[-1],his,randomic_num)

In [10]:
#MAIN GENETIC ALGORITHM
def main():
    parallel_processes = 1 #number of parallel processes that run at the same time, it is equal to the number of individuals in the first generation
    creator.create("FitnessMax", base.Fitness, weights=(1.0,)) #fitness is positive so we maximize it(MCC)
    creator.create("Individual", list, fitness=creator.FitnessMax)
    if __name__ == "__main__":
        #multiprocessing
        pool = multiprocessing.Pool(processes=parallel_processes)
        #create deap tools
        toolbox = base.Toolbox()
        toolbox.register("map", pool.map)
        toolbox.register("evaluate", evaluate_medium)
        toolbox.register("select", tools.selBest, k = 5) #selection algorithm selects the indivudial based on the best fitness
        toolbox.register("individual_creator", prules) #individual creator based on the rules 
        toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.individual_creator, 1)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual) #population takes individual creator function and repeats it multiple times to create the population for the GA
        #initialize population
        individuals_list = []
        generations = []
        fitnesses_list = []
        pop = toolbox.population(n=parallel_processes)
        for g in range(20): #generations
            start_time = time.time()
            fitnesses=toolbox.map(toolbox.evaluate, pop) #test the indivduals by running one instance of the NN network
            for ind, fit in zip(pop, fitnesses): # manually register the fitness due to issues with DEAP classes
                ind.fitness.values = (fit[0],)
                fitnesses_list.append(fit)
                individuals_list.append(ind)
                generations.append(g)
            best_individuals = toolbox.select(pop) #selection of best individuals
            print("BEST INDIVS")
            print(best_individuals)
            offsprings = best_individuals[:]
            for i in range(15): #generate offpring by cloning best individuals and mutating them; no crossover
                offspring = random.choice(best_individuals)
                offspring = toolbox.clone(offspring)
                offspring = mutate_medium(offspring)
                offsprings.append(offspring)
            pop = offsprings[:]
        pool.close()
        return individuals_list,fitnesses_list,generations 

In [9]:
#Load dataset and launch the GA
peptides_df = pd.read_csv("db/DB_hyperfiltered.csv")
funzioni = peptides_df.Function.unique()
for i in funzioni:
    peptides_df = pd.read_csv("db/DB_hyperfiltered.csv")
    print("\n")
    print("Classificazione di ",i)
    peptides_df.loc[peptides_df["Function"] == i, "Class"] = 0 
    functions = peptides_df["Class"]
    best_i, best_f, best_g = main()
    best_res = pd.DataFrame({"Generations" : best_g ,"Indivuduals" : best_i, "Fitnesses":best_f})
    best_res.to_csv(i+".csv")



Classificazione di  Antidiabetic


/home/baccus/.local/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/baccus/.local/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
2023-08-03 14:45:42.082714: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-03 14:45:42.082738: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-03 14:45:42.082764:

(100, 21)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 97, 409)           34765     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 48, 409)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 45, 371)           607327    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 22, 371)          0         
 1D)                                                             
                                                                 
 dropout (Dropout)           (None, 22, 371)           0         
                                                                 
 flatten (Flatten)           (None, 8162)     

KeyboardInterrupt: 

Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/tmp/ipykernel_78173/493337925.py", line 116, in evaluate_medium
    history = model.fit(train_data, a,class_weight=weights, validation_data=(validation_data,c), callbacks=callbacks_list, epochs=50, batch_size=20,use_multiprocessing=False,workers=1,verbose=1)
  File "/home/baccus/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/home/baccus/.local/lib/python3.10/site-packages/keras/engine/training.py", line 16